<a href="https://colab.research.google.com/github/Zahra-FallahMMA/DRL_Offload_Allocation/blob/main/DRL_agent_cost_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Libraries

In [ ]:
import xml.etree.ElementTree as ET
from io import StringIO
import os
import re
import numpy as np
from collections import deque, defaultdict
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from itertools import product
import tensorflow as tf
!pip install simpy
import simpy

# Set TensorFlow logging level to suppress detailed logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Class DQNAgent

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, discount_factor=0.95, exploration_rate=1.0,
                 exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path = None):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        if model_path is not None:
            self.model = load_model(model_path)
        else:
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential()

        # Input layer
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dropout(0.4))  # Slightly reduced dropout to retain more information

        # Hidden layers
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.4))

        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))

        # Output layer
        model.add(Dense(self.action_size, activation='linear'))

        # Compile the model with a custom learning rate scheduler
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def choose_action(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        state = np.array(state).reshape(1, -1)  # Ensure state is 2D
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def save_model(self, path):
        self.model.save(path)

# class Task

In [ ]:
class Task:
    def __init__(self, id, instructions, workflow_id):
        self.id = id
        self.instructions = instructions  # Execution time or computational instructions
        self.children = []  # List of tasks that depend on this task
        self.parents = []  # List of tasks this task depends on

        self.input_files = []   # list of (filename, size)
        self.output_files = []  # list of (filename, size)

        self.executed = False  # Status of execution
        self.executed_on = None  # Node this task was executed on
        self.execution_time = 0  # Time taken to execute the task
        self.cost = 0  # Cost of executing the task
        self.comm_delay = 0  # Communication delay in seconds
        self.workflow_id = workflow_id  # Workflow identifier to which this task belongs
        self.total_transfer_files_size = 0


# Class Workflow

In [ ]:
class Workflow:
    def __init__(self, id):
        self.id = id  # Workflow identifier
        self.tasks = {}  # Dictionary of tasks in the workflow

    def add_task(self, task_id, instructions, parent_ids=[], input_files=[], output_files=[]):
        if task_id not in self.tasks:
            self.tasks[task_id] = Task(task_id, instructions, self.id)
        task = self.tasks[task_id]
        for parent_id in parent_ids:
            if parent_id not in self.tasks:
                self.tasks[parent_id] = Task(parent_id, 0, self.id)
            parent_task = self.tasks[parent_id]
            parent_task.children.append(task)
            task.parents.append(parent_task)

        task.input_files.extend(input_files)
        task.output_files.extend(output_files)

# class parse_dax

In [ ]:
def parse_dax(file_path, workflow_id):
    tree = ET.parse(file_path)
    root = tree.getroot()

    workflow_id = workflow_id
    workflow = Workflow(workflow_id)

    # Pegasus DAX namespace
    dax_ns = '{http://pegasus.isi.edu/schema/DAX}'

    # Parse jobs
    jobs = {job.attrib['id']: job for job in root.findall(f'{dax_ns}job')}

    # Add jobs to workflow
    for job_id, job_elem in jobs.items():
        instructions = float(job_elem.attrib.get('runtime', 0))

        # NEW: Gather input/output files from <uses link="input" ...> or <uses link="output" ...>
        input_files = []
        output_files = []
        for uses_elem in job_elem.findall(f'.//{dax_ns}uses'):
            link_type = uses_elem.attrib.get('link', '')
            file_name = uses_elem.attrib.get('file', '')
            file_size_str = uses_elem.attrib.get('size', '0')
            file_size = int(file_size_str)

            if link_type == 'input':
                input_files.append((file_name, file_size))  # e.g. ("FFI_0_2_subfx.sgt", 262360916)
            elif link_type == 'output':
                output_files.append((file_name, file_size))

        workflow.add_task(job_id,
                          instructions,
                          parent_ids=[],  # We'll handle parents in a moment,
                          input_files=input_files,
                          output_files=output_files)

    # Parse dependencies
    for child in root.findall(f'{dax_ns}child'):
        child_id = child.attrib['ref']
        parent_ids = [parent.attrib['ref'] for parent in child.findall(f'{dax_ns}parent')]
        workflow.add_task(child_id, 0, parent_ids)  # Adds a child node with its parent nodes, setting instructions to 0 to avoid overwrite

    return workflow


def ensemble_of_workflows(name, size=10, distribution='constant', dax_path=''):
    ws = []
    ensemble = []
    directory_path = dax_path  # Directory containing DAX files

    # List and filter files in directory
    files = os.listdir(directory_path)
    filtered_files = [file for file in files if name in file]

    if distribution == 'constant':
        pattern = r'100(?!\d)'
        for s in filtered_files:
            if re.search(pattern, s):
                ensemble = [s] * size  # Replicate the matched file 'size' times
                break
    else:
        numbers = np.random.randint(0, len(filtered_files), size)
        ensemble = [filtered_files[i] for i in numbers]  # Select random files based on uniform distribution
    w_id = 0
    for name in ensemble:
        ws.append(parse_dax(dax_path+name,w_id))
        w_id = w_id + 1

    return ws

# Loading dax files

In [ ]:
from google.colab import drive
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Zahra/dax/'

Mounted at /content/drive


# class Device

In [ ]:
class Device:
    def __init__(self, id, mips, cost_per_hour, env):
        self.id = id
        self.mips = mips
        self.cost_per_hour = cost_per_hour
        self.queue = deque()
        self.runnig_queue = deque()
        self.resource = simpy.Resource(env, capacity=1)

    def add_task_to_queue(self, task):
        self.queue.append(task)

    def get_next_task(self):
        return self.queue.popleft() if self.queue else None

    def waiting_time(self):
        waiting_time = 0
        for t in self.queue:
            waiting_time += t.instructions / self.mips
        return waiting_time


# class FogEnv

In [ ]:
import simpy

class FogEnv:
    def __init__(self, env, iot_devices, fog_nodes, cloud_servers, workflows):
        self.env = env
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.cost = 0
        self.completed_workflows = 0
        self.workflows = workflows
        self.total_delay = 0

        # NEW: Track file origins
        self.file_origins = {}  # { file_name: device_id }

        # Example bandwidth table (Mb/s)
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000,
            ('iot', 'fog'): 500,
            ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500,
            ('fog', 'fog'): 4000,
            ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500,
            ('cloud', 'fog'): 1000,
            ('cloud', 'cloud'): 1000
        }

    def assign_task(self, task, device):
        with device.resource.request() as req:
            yield req

            # 1) Transfer each input file if needed
            for file_name, file_size_bytes in task.input_files:
                self.handle_file_transfer(task, file_name, file_size_bytes, device)
                task.total_transfer_files_size += file_size_bytes

            # 2) Execute the task on this device
            execution_time = task.instructions / device.mips
            self.total_delay += execution_time
            task.execution_time = execution_time
            yield self.env.timeout(execution_time)

            # 3) Update cost
            task.cost = execution_time * device.cost_per_hour
            self.cost += task.cost
            task.executed = True
            task.executed_on = device.id
            task.execution_time = execution_time

            # 4) Mark output files as originating from this device
            for out_file, _sz in task.output_files:
                self.file_origins[out_file] = device.id

            # 5) Check if this workflow is now complete
            wf = next(w for w in self.workflows if w.id == task.workflow_id)
            self.check_workflow_completion(wf)
            device.queue.popleft()

    # NEW: Function to handle transfer
    def handle_file_transfer(self,task, file_name, file_size_bytes, destination_device):
        # If we don't know the file origin, assume it's "local" or from some default place
        if file_name not in self.file_origins:
            return  # no transfer needed

        source_dev_id = self.file_origins[file_name]
        if source_dev_id == destination_device.id:
            return  # already local

        # Get device types for bandwidth lookup
        source_dev = self.get_device_by_id(source_dev_id)
        if source_dev is None:
            return  # fallback: no known device

        source_type = source_dev.dev_type
        dest_type = destination_device.dev_type

        # bandwidth_matrix gives us Mb/s. Convert bytes -> megabits
        # 1 byte = 8 bits, so:
        file_megabits = file_size_bytes * 8 / 1e6

        # Look up or fallback
        band_mbps = self.bandwidth_matrix.get((source_type, dest_type), 1000.0)

        # Transfer time (seconds)
        transfer_time = file_megabits / band_mbps

        # Optional: add base communication latency
        comm_latency = 0.01  # 10 ms, for example
        total_delay = comm_latency + transfer_time
        self.total_delay += total_delay
        task.comm_delay = total_delay

        # Wait out the transfer in simulation time
        yield self.env.timeout(total_delay)

    def get_state(self, task):
        return [self.cost] + [d.waiting_time() for d in self.iot_devices + self.fog_nodes + self.cloud_servers] + [task.instructions, task.cost]

    def get_device_by_id(self, device_id):
        for device in self.iot_devices + self.fog_nodes + self.cloud_servers:
            if device.id == device_id:
                return device
        return None

    def check_workflow_completion(self, workflow):
        # Check if all tasks in the workflow are executed
        if all(task.executed for task in workflow.tasks.values()):
            self.completed_workflows += 1  # Increment completed workflows counter
            # print(f"Workflow {workflow.id} is completed! Total completed workflows: {self.completed_workflows}")





def process_workflow(env, workflow, fog_env, agent):
      while(True):
        if all([task.executed for task in workflow.tasks.values()]):
          break
        for task in workflow.tasks.values():
            if task.executed:
              continue
            if all([parent.executed for parent in task.parents]) or task.parents == []:
                state = np.array(fog_env.get_state(task)).reshape(1, -1)
                action = agent.choose_action(state)
                devices = fog_env.iot_devices + fog_env.fog_nodes + fog_env.cloud_servers
                device = devices[action]
                device.add_task_to_queue(task)
                task.executed_on = device.id
                yield env.process(fog_env.assign_task(task, device))

# class NewSim



In [ ]:
class  NewSim:
    def __init__(self, num_iot, num_fog, num_server, learning_rate=0.001, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path=None):
        self.num_iot = num_iot
        self.num_fog = num_fog
        self.num_server = num_server
        self.num_totla_dev = num_iot + num_fog + num_server
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model_path = model_path
        self.env = simpy.Environment()
        self.reset()
        self.run()

    def reset(self):
        self.iot_devices = [Device(f'iot_{i}', 500, 0, self.env) for i in range(self.num_iot)]
        self.fog_devices = [Device(f'fog_{i}', 4000, 1/3600, self.env) for i in range(self.num_fog)]
        self.server_devices = [Device(f'server_{i}', 6000, 8/3600, self.env) for i in range(self.num_server)]
        self.agent = DQNAgent(state_size=3+self.num_totla_dev, action_size=self.num_totla_dev, learning_rate=self.learning_rate, discount_factor=self.discount_factor,
                                  exploration_rate=self.exploration_rate, exploration_decay=self.exploration_decay,
                                  exploration_min=self.exploration_min, batch_size=self.batch_size, memory_size=self.memory_size, model_path = self.model_path)

        self.workflows = ensemble_of_workflows(name = 'CyberShake', size=100, distribution = 'constant', dax_path="/content/drive/My Drive/Zahra/dax/")
    def run(self):
        self.fog_env = FogEnv(self.env, self.iot_devices, self.fog_devices, self.server_devices,self.workflows)
        for workflow in self.workflows:
            self.env.process(process_workflow(self.env, workflow, self.fog_env, self.agent))

        self.env.run()  # Run simulation for a time period


# Data Collection


## The Config of Workflows

In [ ]:
# Workflow configurations with specific sizes
workflow_configs = {
        'CyberShake': [30, 50, 100, 1000],
        'Montage': [20, 40, 60, 80, 100, 200, 300],
        'Inspiral': [30, 50, 100, 1000],
        'Sipht': [29, 58, 100, 968]
    }
workflow_distributions = ['constant', 'uniform']

In [ ]:
def run_simulation_with_results_tracking(workflow_name, workflow_size, workflow_distribution, model_path):
    learning_rate = 0.001
    discount_factor = 0.99
    exploration_rate = 0.5
    exploration_decay = 0.995
    exploration_min = 0.05

    dax_path = "/content/drive/My Drive/Zahra/dax/"

    print(f"Running simulation for Workflow: {workflow_name}, Size: {workflow_size}, Distribution: {workflow_distribution}")

    # Set up the simulation with the current parameters
    simulation = NewSim(
        num_iot=10,
        num_fog=8,
        num_server=5,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        exploration_rate=exploration_rate,
        exploration_decay=exploration_decay,
        exploration_min=exploration_min,
        model_path = model_path
    )

    # Update the workflow parameters
    simulation.workflows = ensemble_of_workflows(
        name=workflow_name,
        size=workflow_size,
        distribution=workflow_distribution,
        dax_path=dax_path
    )

    # Run the simulation
    simulation.run()

    # Print results for current run
    print(f"Total cost for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.fog_env.cost}")
    print(f"Total time for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.env.now}")




# Results

## Cybershake - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 0.002664614583333338
Total time for Workflow CyberShake (constant, size=10): 46.813854999999414
Running simulation for Workflow: CyberShake, Size: 20, Distribution: constant
Total cost for Workflow CyberShake (constant, size=20): 0.005385139583333338
Total time for Workflow CyberShake (constant, size=20): 50.67137583333272
Running simulation for Workflow: CyberShake, Size: 30, Distribution: constant
Total cost for Workflow CyberShake (constant, size=30): 0.008646326851851863
Total time for Workflow CyberShake (constant, size=30): 55.060958333332735
Running simulation for Workflow: CyberShake, Size: 40, Distribution: constant
Total cost for Workflow CyberShake (constant, size=40): 0.011391696296296352
Total time for Workflow CyberShake (constant, size=40): 59.72076999999942
Running simulation for Workflow: CyberShake, Size: 50, Distribution: constant
Tota

## Cybershake - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=10): 0.0072710087962963065
Total time for Workflow CyberShake (uniform, size=10): 62.85784333333267
Running simulation for Workflow: CyberShake, Size: 20, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=20): 0.009708684953703695
Total time for Workflow CyberShake (uniform, size=20): 64.87280999999955
Running simulation for Workflow: CyberShake, Size: 30, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=30): 0.011015264120370382
Total time for Workflow CyberShake (uniform, size=30): 67.11173583333303
Running simulation for Workflow: CyberShake, Size: 40, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=40): 0.026068413425925818
Total time for Workflow CyberShake (uniform, size=40): 83.44226333333341
Running simulation for Workflow: CyberShake, Size: 50, Distribution: uniform
Total cost for Wor

## Montage - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: constant
Total cost for Workflow Montage (constant, size=10): 0.0009048701388888907
Total time for Workflow Montage (constant, size=10): 44.728559999999455
Running simulation for Workflow: Montage, Size: 20, Distribution: constant
Total cost for Workflow Montage (constant, size=20): 0.0018953946759259163
Total time for Workflow Montage (constant, size=20): 45.146189166666076
Running simulation for Workflow: Montage, Size: 30, Distribution: constant
Total cost for Workflow Montage (constant, size=30): 0.002775119212962956
Total time for Workflow Montage (constant, size=30): 46.06089749999975
Running simulation for Workflow: Montage, Size: 40, Distribution: constant
Total cost for Workflow Montage (constant, size=40): 0.003789247453703693
Total time for Workflow Montage (constant, size=40): 48.295861666666354
Running simulation for Workflow: Montage, Size: 50, Distribution: constant
Total cost for Workflow Montage (constan

## Montage - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: uniform
Total cost for Workflow Montage (uniform, size=10): 0.0009913289351851848
Total time for Workflow Montage (uniform, size=10): 43.7207116666662
Running simulation for Workflow: Montage, Size: 20, Distribution: uniform
Total cost for Workflow Montage (uniform, size=20): 0.001758653935185175
Total time for Workflow Montage (uniform, size=20): 45.43997916666616
Running simulation for Workflow: Montage, Size: 30, Distribution: uniform
Total cost for Workflow Montage (uniform, size=30): 0.002645554861111101
Total time for Workflow Montage (uniform, size=30): 47.04092249999955
Running simulation for Workflow: Montage, Size: 40, Distribution: uniform
Total cost for Workflow Montage (uniform, size=40): 0.003987087037037031
Total time for Workflow Montage (uniform, size=40): 49.154427499999514
Running simulation for Workflow: Montage, Size: 50, Distribution: uniform
Total cost for Workflow Montage (uniform, size=50): 0.005

## Sipht - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: constant
Total cost for Workflow Sipht (constant, size=10): 0.01922214177546299
Total time for Workflow Sipht (constant, size=10): 71.24770128333293
Running simulation for Workflow: Sipht, Size: 20, Distribution: constant
Total cost for Workflow Sipht (constant, size=20): 0.0356397205972223
Total time for Workflow Sipht (constant, size=20): 96.2186415583332
Running simulation for Workflow: Sipht, Size: 30, Distribution: constant
Total cost for Workflow Sipht (constant, size=30): 0.04992042037268531
Total time for Workflow Sipht (constant, size=30): 111.44615106666664
Running simulation for Workflow: Sipht, Size: 40, Distribution: constant
Total cost for Workflow Sipht (constant, size=40): 0.0623345361435185
Total time for Workflow Sipht (constant, size=40): 137.92299882499984
Running simulation for Workflow: Sipht, Size: 50, Distribution: constant
Total cost for Workflow Sipht (constant, size=50): 0.07790149988888842
Total

## Sipht - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=10): 0.020513466546296318
Total time for Workflow Sipht (uniform, size=10): 158.51422369166642
Running simulation for Workflow: Sipht, Size: 20, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=20): 0.01583474960648147
Total time for Workflow Sipht (uniform, size=20): 84.99369159166635
Running simulation for Workflow: Sipht, Size: 30, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=30): 0.12052681411574083
Total time for Workflow Sipht (uniform, size=30): 320.41795684166624
Running simulation for Workflow: Sipht, Size: 40, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=40): 0.12935208287731526
Total time for Workflow Sipht (uniform, size=40): 327.849485866666
Running simulation for Workflow: Sipht, Size: 50, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=50): 0.16927392703703525
Total time for W

## Inspiral - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: constant
Total cost for Workflow Inspiral (constant, size=10): 0.0193124217592593
Total time for Workflow Inspiral (constant, size=10): 68.76886083333274
Running simulation for Workflow: Inspiral, Size: 20, Distribution: constant
Total cost for Workflow Inspiral (constant, size=20): 0.03517783032407392
Total time for Workflow Inspiral (constant, size=20): 97.22700499999908
Running simulation for Workflow: Inspiral, Size: 30, Distribution: constant
Total cost for Workflow Inspiral (constant, size=30): 0.05737474652777758
Total time for Workflow Inspiral (constant, size=30): 131.98726416666543
Running simulation for Workflow: Inspiral, Size: 40, Distribution: constant
Total cost for Workflow Inspiral (constant, size=40): 0.07440021666666663
Total time for Workflow Inspiral (constant, size=40): 150.61606416666626
Running simulation for Workflow: Inspiral, Size: 50, Distribution: constant
Total cost for Workflow Inspiral (c

## Inspiral - uniform

In [15]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: uniform
Total cost for Workflow Inspiral (uniform, size=10): 0.050823355092592566
Total time for Workflow Inspiral (uniform, size=10): 207.10183416666598
Running simulation for Workflow: Inspiral, Size: 20, Distribution: uniform
Total cost for Workflow Inspiral (uniform, size=20): 0.1328519749999999
Total time for Workflow Inspiral (uniform, size=20): 299.5527183333331
Running simulation for Workflow: Inspiral, Size: 30, Distribution: uniform
Total cost for Workflow Inspiral (uniform, size=30): 0.20616560694444555
Total time for Workflow Inspiral (uniform, size=30): 370.74577916666624
Running simulation for Workflow: Inspiral, Size: 40, Distribution: uniform
Total cost for Workflow Inspiral (uniform, size=40): 0.20801623125000038
Total time for Workflow Inspiral (uniform, size=40): 407.3264133333338
Running simulation for Workflow: Inspiral, Size: 50, Distribution: uniform
Total cost for Workflow Inspiral (uniform, size